In [1]:
!nvidia-smi

Tue Dec 15 10:09:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# download and extract dataset from google drive link
# dataset link: https://drive.google.com/uc?id=1oj7HdETwTZUMVzR-781uwK8WleCHqniN
%cd /content/
!gdown --id 1oj7HdETwTZUMVzR-781uwK8WleCHqniN
!unzip /content/mpii_pushup.zip

Streaming output truncated to the last 5000 lines.
  inflating: mpii_pushup/images/mpii_crop_9687.png  
  inflating: mpii_pushup/images/mpii_crop_9690.png  
  inflating: mpii_pushup/images/mpii_crop_9865.png  
  inflating: mpii_pushup/images/106_200.png  
  inflating: mpii_pushup/images/147_63.png  
  inflating: mpii_pushup/images/153_110.png  
  inflating: mpii_pushup/images/153_1110.png  
  inflating: mpii_pushup/images/17_1520.png  
  inflating: mpii_pushup/images/19_1970.png  
  inflating: mpii_pushup/images/206_660.png  
  inflating: mpii_pushup/images/213_220.png  
  inflating: mpii_pushup/images/267_38.png  
  inflating: mpii_pushup/images/290_260.png  
  inflating: mpii_pushup/images/294_31.png  
  inflating: mpii_pushup/images/403_79.png  
  inflating: mpii_pushup/images/418_25.png  
  inflating: mpii_pushup/images/452_300.png  
  inflating: mpii_pushup/images/452_390.png  
  inflating: mpii_pushup/images/452_480.png  
  inflating: mpii_pushup/images/458_590.png  
  inflating:

In [4]:
# download and extract pre-trained models for testing
%cd /content/
!gdown --id 1FdkHq03rr-tH6Kep-wjlHGp1hb27bbrA
!unzip /content/DL_ckps.zip

/content
Downloading...
From: https://drive.google.com/uc?id=1FdkHq03rr-tH6Kep-wjlHGp1hb27bbrA
To: /content/DL_ckps.zip
52.6MB [00:00, 197MB/s]
Archive:  /content/DL_ckps.zip
  inflating: DL_ckps/mobilenet-separable-deconv-deeper_1012_epoch32.pt  
  inflating: DL_ckps/resnest-separable-deconv-deeper_newdata_1212.pt  
  inflating: DL_ckps/shufflenet-separable-deconv-deeper_epoch60.pt  


In [5]:
!pip install efficientnet_pytorch
!pip install resnest --pre
!pip install albumentations==0.4.5

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16030 sha256=3ccaa68a75e0d69fdfd29dd1b0449eac0779dbdd76b4130195d8b78e0b07ef5b
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch
     |████████████████████████████████| 163kB 6.6MB/s 
     |████████████████████████████████| 122kB 4.1MB/s 
     |████████████████████████████████| 634kB 5.6MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=905cf7373ff6bfa6ab03bc9b11d6bab8ca761ea4bc9aedf7bc92e5cb5bd034a9
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=8d5a2a096233949ee989c74db4df7e11bd9888ab32f286372d2f532629b15325
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8

In [1]:
%cd /content/drive/MyDrive/SHPE_pushup
from loss_funcs import Detection_based_Loss, Regression_based_Loss, MAE, PCKS
from deconv_model import build_model
# import SHPE_pushup.utils
from models import SHPE_model

/content/drive/MyDrive/SHPE_pushup


In [1]:
import os, shutil, cv2, zipfile, json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import mobilenet_v2, shufflenet_v2_x1_5, shufflenet_v2_x0_5, shufflenet_v2_x1_0
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from albumentations import Compose, Resize, HorizontalFlip, VerticalFlip, Blur, MotionBlur, OneOf, GaussNoise, IAAAdditiveGaussianNoise, ShiftScaleRotate, RandomBrightnessContrast
from albumentations.pytorch import ToTensor
from albumentations import KeypointParams
from PIL import Image

In [2]:
class finetune_data(Dataset):
  def __init__(self, labels_json, folder='./mpii_pushup/images', stride=4, img_size=(224,224), transform=None, sub_trans=None, mode = 'train'):
    self.transform = transform
    self.sub_trans = sub_trans
    self.mode = mode
    with open(labels_json, 'r') as fr:
      raw_labels = json.load(fr)
    self.labels = [aa for aa in raw_labels if len(aa['points']) == 7]
    self.folder = folder
    self.n_data = len(self.labels)
    self.stride = stride
    self.img_size = img_size
    self.mask_size = (img_size[0]//stride, img_size[1]//stride)
    print('total length data: {}'.format(self.n_data))
  def __len__(self):
    return self.n_data
  def __getitem__(self, idx):
    label = self.labels[idx]
    img_path = os.path.join(self.folder, label['image'])
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    o_h, o_w = img.shape[:2]
    masks = np.zeros((self.mask_size[0], self.mask_size[1], 3*(len(label['points']))), dtype = np.float32)
    raw_label = np.array(label['points'])
    if o_h > o_w:
      new_img = np.zeros((o_h, o_h, 3), np.uint8)
      cl = ((o_h-o_w)//2)
      new_img[:,cl:cl+o_w] = img.copy()
      raw_label[:,0] = raw_label[:,0] + cl
    else:
      new_img = np.zeros((o_w, o_w, 3), np.uint8)
      cl = ((o_w-o_h)//2)
      new_img[cl:cl+o_h,:] = img.copy()
      raw_label[:,1] = raw_label[:,1] + cl
    if self.transform:
      if self.mode == 'train':
        loop = True
        count = 0
        while loop:
          aug = self.transform(image = new_img, keypoints = raw_label)
          nimg = aug['image']
          ncoors = aug['keypoints']
          nncoors = np.array(ncoors)/self.img_size
          count += 1
          if np.all(nncoors<1) and np.all(nncoors>=0):
            img=nimg
            raw_label=np.array(ncoors).astype(np.int32)
            loop=False
            break
          elif count > 20:
            aug = self.sub_trans(image = new_img)
            img = aug['image']
            raw_label = (np.array(raw_label)/max([o_w, o_h]))*self.img_size
            break
      else:
        aug = self.transform(image = new_img)
        img = aug['image']
        raw_label = (np.array(raw_label)/max([o_w, o_h]))*self.img_size
    for i,key_point in enumerate(raw_label):
        coor_x = np.clip(int(key_point[0]), 0, self.img_size[1]-1)
        coor_y = np.clip(int(key_point[1]), 0, self.img_size[0]-1)
        os_vect_x = coor_x - (coor_x//self.stride)*self.stride
        os_vect_y = coor_y - (coor_y//self.stride)*self.stride
        masks[coor_y//self.stride, coor_x//self.stride, i] = 1.0
        masks[coor_y//self.stride, coor_x//self.stride, i+7] = os_vect_x/self.mask_size[1]
        masks[coor_y//self.stride, coor_x//self.stride, i+14] = os_vect_y/self.mask_size[0]
    return img, transforms.ToTensor()(masks)

In [3]:
%cd /content/
train_json = './mpii_pushup/train.json'
val_json = './mpii_pushup/val.json'
test_json = './mpii_pushup/test.json'
folder='./mpii_pushup/images'
img_size = (224,224)
stride = 4
batch_size = 32

/content


In [4]:
sub_trans = Compose([Resize(img_size[0], img_size[1]), ToTensor()])
trans = Compose([
                 Resize(img_size[0], img_size[1]), #HorizontalFlip(), 
                 RandomBrightnessContrast(always_apply=True, brightness_limit=0.25, contrast_limit=0.25),
                 OneOf([Blur(blur_limit=2, p=0.6), MotionBlur(blur_limit=3, p=0.6)], p=0.6), IAAAdditiveGaussianNoise(scale=(0.01*255, 0.03*255), p=0.6),
                 ShiftScaleRotate(0.0, 0.05, 5, always_apply=True),
                 ToTensor(),], keypoint_params=KeypointParams(format='xy', remove_invisible=False))
data_train = finetune_data(train_json, folder = folder, transform=trans, stride=stride, img_size=img_size, sub_trans=sub_trans)
data_val = finetune_data(val_json, folder = folder, transform=sub_trans, stride=stride, img_size=img_size, mode='val')
data_test = finetune_data(test_json, folder = folder, transform=sub_trans, stride=stride, img_size=img_size, mode='test')
train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(data_val, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=4)

total length data: 18324
total length data: 2188
total length data: 2011


In [ ]:
for i in range(50):
  rand_id = np.random.randint(0,data_val.__len__(),1)[0]
  a, b = data_train.__getitem__(rand_id)
  img_n = (a.numpy()*255).astype(np.uint8)
  masks = b.numpy()
  coor_x = []
  coor_y = []
  for i,mask in enumerate(masks[:7]):
    cx = np.argmax(mask)%mask.shape[0]
    cy = np.argmax(mask)//mask.shape[0]
    ovx = int(masks[i+7][cy,cx]*stride)
    ovy = int(masks[i+14][cy,cx]*stride)
    coor_x.append(cx*stride+ovx)
    coor_y.append(cy*stride+ovy)
  plt.figure(i, figsize=(6,6))
  plt.axis('off')
  plt.imshow(np.stack([img_n[0,...], img_n[1,...], img_n[2,...]], axis=-1))
  plt.scatter(coor_x, coor_y)
  plt.show()

In [17]:
# model_name = 'shuffle'
model_name = 'resnest2'
# model_name = 'mobile2'
model = build_model(model_name, use_depthwise=True)

metrics = {'pcks':PCKS('detection', 7, (224, 224), stride=stride, thresh=0.25, id_shouder=(2,4)), 'mae':MAE('detection', 7, (224, 224), stride=stride)}
criterion = Detection_based_Loss(n_kps=7)
opt = torch.optim.Adam
lr=3e-4

shpe_model = SHPE_model(loss_func= criterion, optimizer=opt, lr=lr, pb_type='define', model_name='resnest',
                        metrics=metrics, define_model=model, define_img_size=(224,224), stride=stride, n_kps=7)

In [18]:
shpe_model.load_ckp('/content/DL_ckps/resnest-separable-deconv-deeper_newdata_1212.pt')
# shpe_model.load_ckp('/content/DL_ckps/mobilenet-separable-deconv-deeper_1012_epoch32.pt')
# shpe_model.load_ckp('/content/DL_ckps/shufflenet-separable-deconv-deeper_epoch60.pt')

In [19]:
shpe_model.evaluate(val_loader)
shpe_model.evaluate(test_loader)

pcks: 0.793 - mae: 0.028 - loss: 0.006 
pcks: 0.818 - mae: 0.026 - loss: 0.006 


{'loss': 0.005974888581363924,
 'mae': 0.02556920958722722,
 'pcks': 0.8180010328833467}

In [16]:
!pip install torch-summary
from torchsummary import summary
summary(model)

Layer (type:depth-idx)                                  Param #
├─Sequential: 1-1                                       --
|    └─ConvBNReLU: 2-1                                  --
|    |    └─Conv2d: 3-1                                 864
|    |    └─BatchNorm2d: 3-2                            64
|    |    └─ReLU6: 3-3                                  --
|    └─InvertedResidual: 2-2                            --
|    |    └─Sequential: 3-4                             896
├─Sequential: 1-2                                       --
|    └─InvertedResidual: 2-3                            --
|    |    └─Sequential: 3-5                             5,136
|    └─InvertedResidual: 2-4                            --
|    |    └─Sequential: 3-6                             8,832
├─Sequential: 1-3                                       --
|    └─InvertedResidual: 2-5                            --
|    |    └─Sequential: 3-7                             10,000
|    └─InvertedResidual: 2-6           

Layer (type:depth-idx)                                  Param #
├─Sequential: 1-1                                       --
|    └─ConvBNReLU: 2-1                                  --
|    |    └─Conv2d: 3-1                                 864
|    |    └─BatchNorm2d: 3-2                            64
|    |    └─ReLU6: 3-3                                  --
|    └─InvertedResidual: 2-2                            --
|    |    └─Sequential: 3-4                             896
├─Sequential: 1-2                                       --
|    └─InvertedResidual: 2-3                            --
|    |    └─Sequential: 3-5                             5,136
|    └─InvertedResidual: 2-4                            --
|    |    └─Sequential: 3-6                             8,832
├─Sequential: 1-3                                       --
|    └─InvertedResidual: 2-5                            --
|    |    └─Sequential: 3-7                             10,000
|    └─InvertedResidual: 2-6           